# Texture and hyperchromatic areas feature exploration

In [1]:
import os
import cv2
import random
import numpy as np
import matplotlib.pyplot as plt
from skimage.segmentation import flood_fill
from skimage import io, color, exposure, img_as_ubyte
from skimage.feature import graycomatrix, graycoprops
from sklearn.cluster import KMeans, MeanShift

In [2]:
#Uncomment to install dependecy
#!pip install opencv-python 

In [3]:
#pip install scikit-image

### Declare variables and lists

In [3]:
# Images root directorie
data_dir = "cancerSeno_bw/train"

# Class list
classes = ["sano", "cancer"]

# Define angles and distanxes for GLCM (Gray-Level Co-Occurrence Matrix)
angles = [0]
distances = [1]

# Define normalization threshold
treshold = 120

# Define max number of clusters for segmentation
max_clusters = 5

# Initialize lists
# Store texture and density of pixels characteristics
glcm_features = []
densidades_pixeles_oscuros = []
num_cúmulos = []

# Store coordinates of origins in clusters
cluster_seeds = []

# Store loaded images
loaded_images = []

### Obtain images from directories

In [4]:
# Iterate over classes





health_dir = os.path.join(data_dir, classes[0])
ill_dir = os.path.join(data_dir, classes[1])
# Obtain list of image files from directory
health_file_list = os.listdir(health_dir)
ill_file_list = os.listdir(ill_dir)
    
# Store the first 3 images from current class
health_images_source = [os.path.join(health_dir, image_name) for image_name in health_file_list]
ill_images_source = [os.path.join(ill_dir, image_name) for image_name in ill_file_list]
    
# Add image paths to the loaded-images list
#loaded_images.extend(selected_images)

health_images = np.array([np.array(io.imread(image_path)) for image_path in health_images_source])
ill_images = np.array([np.array(io.imread(image_path)) for image_path in ill_images_source])


print(len(health_images))
print(len(ill_images))

C:\Users\Lenovo\anaconda3\envs\entorno3.7\lib\site-packages\ipykernel_launcher.py:20: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


185917
73554


C:\Users\Lenovo\anaconda3\envs\entorno3.7\lib\site-packages\ipykernel_launcher.py:21: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


In [7]:
health_images

array([array([[187, 180, 194, ..., 229, 202, 224],
              [170, 163, 177, ..., 191, 199, 221],
              [200, 183, 159, ..., 199, 169, 188],
              ...,
              [172, 161, 160, ..., 151, 167, 154],
              [172, 160, 153, ..., 104, 140, 142],
              [157, 164, 184, ..., 108, 185, 194]], dtype=uint8),
       array([[176, 181, 193, ..., 179, 214, 208],
              [177, 170, 163, ..., 171, 175, 162],
              [154, 171, 186, ..., 165, 152, 151],
              ...,
              [239, 248, 247, ..., 242, 246, 237],
              [245, 242, 244, ..., 247, 245, 247],
              [214, 246, 247, ..., 246, 249, 247]], dtype=uint8),
       array([[246, 247, 248, ..., 249, 247, 246],
              [247, 246, 249, ..., 245, 247, 249],
              [235, 231, 239, ..., 246, 247, 245],
              ...,
              [248, 248, 248, ..., 243, 232, 181],
              [248, 248, 248, ..., 245, 207, 133],
              [248, 248, 248, ..., 248, 220, 1

In [9]:

import pickle
with open('gray_health_images_vector.pickle','wb') as file:
    pickle.dump(health_images, file)

In [10]:
ill_images

array([array([[253, 252, 252, ..., 251, 251, 251],
              [253, 252, 252, ..., 251, 251, 251],
              [252, 252, 252, ..., 251, 251, 251],
              ...,
              [252, 252, 252, ...,  85,  72, 102],
              [252, 252, 251, ...,  87,  73,  69],
              [252, 252, 251, ..., 109, 106, 117]], dtype=uint8),
       array([[252, 252, 252, ..., 250, 196,  67],
              [252, 252, 252, ..., 248, 117, 128],
              [252, 252, 252, ..., 120, 129, 102],
              ...,
              [251, 251, 251, ..., 112, 116,  76],
              [251, 251, 251, ...,  66, 100, 161],
              [251, 251, 251, ..., 114, 242, 248]], dtype=uint8),
       array([[251, 251, 251, ..., 205, 249, 250],
              [251, 251, 250, ..., 233, 248, 249],
              [251, 251, 251, ..., 187, 249, 249],
              ...,
              [107,  99, 132, ..., 126, 210, 247],
              [ 72,  64, 125, ..., 107,  83, 123],
              [ 85,  91,  89, ..., 142, 124,  

In [11]:
with open('gray_ill_images_vector.pickle','wb') as file:
    pickle.dump(ill_images, file)